Many-Body Chern Number without Integration

Koji Kudo, Haruki Watanabe, Toshikaze Kariyado, and Yasuhiro Hatsugai

Phys. Rev. Lett. 122, 146601 – Published 9 April 2019

https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.122.146601

In [82]:
LL = [3, 3]
N = LL[1]*LL[2]
vv_lattice = [ [1., 0], [0, 1.] ]
b1=[0, 0]
vv_basis = [b1]
t=-1
p=1
q=3
ϕ=p/q
pbc=true
param_dict=Dict{}()
param_dict[t]=t
param_dict[ϕ]=ϕ;

In [83]:
using NBInclude
@nbinclude("../../../Single Particle/Lattice_Basis_Neighbors_Hops.ipynb"; regex=r"#.*executeme")

get_Hofstadter_hamiltonian (generic function with 2 methods)

In [84]:
using QuantumOptics

## Initial Parameters

In [85]:
N1 = 10
N2 = 10
dx=2*pi/N1
dy=dx
Tx=collect(range(start=0, stop=2*pi-dx, step=dx))
Ty=collect(range(start=0, stop=2*pi-dy, step=dy));

## Create Single-Particle Operator

In [86]:
function Op_Sp(sites,neighbors_list,
               site_number,site_coord,
               nb,LL,param_dict,tx,ty,pbc)
    
    #H_T = HSP_T(Nx, Ny, alpha, Tx, Ty, 0)
    H_T = get_Hofstadter_hamiltonian(sites,neighbors_list,
                                    site_number,site_coord,
                                    length(vv_basis),LL,param_dict,
                                    tx,ty,pbc)
    N = LL[1]*LL[2]
    basis = NLevelBasis(N) 
    
    H = SparseOperator(basis)
    
    for m in 1:N
        for n in 1:N
            H = H + H_T[m,n] * transition(basis, m, n)
        end
    end
    
    return H
end

Op_Sp (generic function with 1 method)

## Create Many-Basis for Non-interacting system

In [90]:
function get_mb_op(PN, op)
    
    basis = NLevelBasis(N)
    states = bosonstates(basis, PN)
    basis_mb = ManyBodyBasis(basis, states)
    
    H = SparseOperator(basis_mb) 
    for m in 1:N
        for n in 1:N
            H += op.data[m,n] * transition(basis_mb, m, n)
        end
    end
    
    return H
end

get_mb_op (generic function with 1 method)

# Hamiltonian

In [91]:
function HH(sites,neighbors_list,
           site_number,site_coord,
           nb,LL,param_dict,tx,ty,pbc,PN)
    
    op = Op_Sp(sites,neighbors_list,
           site_number,site_coord,
           length(vv_basis),LL,param_dict,tx,ty,pbc)
    
    H_mb = get_mb_op(PN, op)
    return H_mb
end

HH (generic function with 1 method)

In [94]:
PN=1
H = HH(sites,neighbors_list,
site_number,site_coord,
length(vv_basis),LL,param_dict,0,0,pbc,PN);

In [64]:
# CHECK: MB Structure (PN=1)
#eigenenergies(dense(H_mb)) == eigvals(HSP_T(Nx, Ny, 1/q, Tx, Ty, 0))
# eigenenergies(dense(get_mb_op(basis_mb, op)))

In [65]:
# # Ortogonality Check !
# E, V = eigen(dense(get_mb_op(basis_mb, op)).data)
# dot(V[:,1],V[:,2])
# norm(V)

## Create Mesh of Twisted-Angle Space

\begin{equation}
\theta_\mu = \frac{2\pi}{N_\theta}n_\mu
\end{equation}

In [66]:
# dx=2*pi/T_size
# dy=dx
# Tx=collect(range(start=0, stop=2*pi-dx, step=dx))
# Ty=collect(range(start=0, stop=2*pi-dy, step=dy));

## Energies of Many-Body Hofstadter Model

In [67]:
# using Plots
# E = eigenenergies(dense(H_mb))
# plot(1:length(E),E,seriestype=:scatter,legend=false)

# Cherns

Berry Curvature:
\begin{equation}
\mathcal{F}(\vec{\theta}) = \log \left( U_x(\vec{\theta})U_y(\vec{\theta}+\delta_x)U_x(\vec{\theta}+\delta_y)^{-1}U_y(\vec{\theta})^{-1} \right)
\end{equation}
Link variables:
\begin{equation}
U_\mu(\vec{\theta}) = \frac{
det \left( \Phi(\vec{\theta})^\dagger \Phi(\vec{\theta}+\delta_\mu) \right) 
}{
| det \left(\Phi(\vec{\theta})^\dagger \Phi(\vec{\theta}+\delta_\mu) \right) |} , \quad \delta_x = \left(\frac{2\pi}{N_{\theta}},0 \right), \delta_y = \left(0, \frac{2\pi}{N_{\theta}}\right)
\end{equation}
Chern Number:
\begin{equation}
\mathcal{C} = \frac{1}{2\pi i} \sum_{\vec{\theta}} \mathcal{F}(\vec{\theta})
\end{equation}

In [68]:
# using LinearAlgebra
# PN=1
# tx = Tx[7]
# ty = Ty[1]
# H_mb = HH(sites,neighbors_list,
#            site_number,site_coord,
#            length(vv_basis),LL,param_dict, tx, ty, pbc,PN)
# E, U = eigen(dense(H_mb).data)
# E
# U

In [96]:
dimH = size(H)[1]

9

In [148]:
@nbinclude("../../Chern_parameter_space_sum.ipynb")

calc_F12 (generic function with 1 method)

In [149]:
EEA, UUA = calc_states_hofstadter(Tx, Ty, dimH)
EEA[1,1,:]
UUA[2,2,:,:]

9×9 Matrix{ComplexF64}:
  -0.227724+0.513248im     0.108487+0.0230596im    …   -0.410703-0.231253im
  -0.131384+0.388706im   -0.0115933+0.110303im         0.0293874+0.403802im
   0.038881+0.316818im      0.35177+0.174995im          0.270478-0.374042im
 -0.0721484+0.427378im   -0.0897285-0.0651916im         0.240697+0.368163im
  -0.184192+0.0613696im   -0.110911-2.51535e-17im      0.0505765-0.122546im
   0.118306-0.0873464im    0.387447+0.0651916im    …   -0.133002+0.332631im
 -0.0386375+0.377225im   -0.0243352-0.392139im         0.0470597-0.206568im
   0.105799+0.110983im    -0.250181+0.302944im        -0.0516903-0.117968im
        0.0+0.0im           0.57735-0.0im                    0.0+0.0im

In [150]:
UUA[1,1,:,1]'*UUA[1,1,:,1]

1.0 + 0.0im

In [151]:
UUA = OffsetArray(UUA, 0:N1-1, 0:N2-1, 0:dimH-1, 0:dimH-1)
UUA[1,1,:,:]

9×9 OffsetArray(::Matrix{ComplexF64}, 0:8, 0:8) with eltype ComplexF64 with indices 0:8×0:8:
  -0.227724+0.513248im     0.108487+0.0230596im    …   -0.410703-0.231253im
  -0.131384+0.388706im   -0.0115933+0.110303im         0.0293874+0.403802im
   0.038881+0.316818im      0.35177+0.174995im          0.270478-0.374042im
 -0.0721484+0.427378im   -0.0897285-0.0651916im         0.240697+0.368163im
  -0.184192+0.0613696im   -0.110911-2.51535e-17im      0.0505765-0.122546im
   0.118306-0.0873464im    0.387447+0.0651916im    …   -0.133002+0.332631im
 -0.0386375+0.377225im   -0.0243352-0.392139im         0.0470597-0.206568im
   0.105799+0.110983im    -0.250181+0.302944im        -0.0516903-0.117968im
        0.0+0.0im           0.57735-0.0im                    0.0+0.0im

In [152]:
calc_link_var(UUA[1,6,:,1], UUA[9,1,:,1])

0.9833994827045871 - 0.18145373354205335im

In [153]:
UU = calc_link_vars(UUA[:,:,:,1:2])

10×10×2 OffsetArray(::Array{ComplexF64, 3}, 0:9, 0:9, 0:1) with eltype ComplexF64 with indices 0:9×0:9×0:1:
[:, :, 0] =
 0.00163509+0.999999im   0.285763+0.9583im     …    0.857324-0.514777im
    0.48143+0.876485im  -0.982476+0.186389im       -0.697065+0.717008im
  -0.978167+0.207821im   0.604138+0.79688im        -0.985228+0.171246im
    0.78589-0.618367im  -0.380902+0.924615im       -0.570605+0.821225im
  -0.972366+0.23346im     0.32284-0.946454im        0.467586+0.883948im
  -0.760918-0.648848im   0.998838-0.0481857im  …   -0.970078-0.242794im
 -0.0914562-0.995809im  -0.853892-0.520451im        -0.25856-0.965995im
  -0.992672+0.120841im  -0.808415+0.588613im        0.279452+0.96016im
   0.980437-0.196833im   0.380447+0.924803im      -0.0076575-0.999971im
   0.955999+0.293371im  -0.955126-0.296201im      -0.0458815+0.998947im

[:, :, 1] =
 -0.921215-0.389054im    0.935704-0.352786im  …  -0.819825+0.572615im
 -0.165691+0.986178im   -0.615573-0.78808im      -0.408947+0.912558im
 0.04592

In [154]:
UU[:,:,0]

10×10 OffsetArray(::Matrix{ComplexF64}, 0:9, 0:9) with eltype ComplexF64 with indices 0:9×0:9:
 0.00163509+0.999999im   0.285763+0.9583im     …    0.857324-0.514777im
    0.48143+0.876485im  -0.982476+0.186389im       -0.697065+0.717008im
  -0.978167+0.207821im   0.604138+0.79688im        -0.985228+0.171246im
    0.78589-0.618367im  -0.380902+0.924615im       -0.570605+0.821225im
  -0.972366+0.23346im     0.32284-0.946454im        0.467586+0.883948im
  -0.760918-0.648848im   0.998838-0.0481857im  …   -0.970078-0.242794im
 -0.0914562-0.995809im  -0.853892-0.520451im        -0.25856-0.965995im
  -0.992672+0.120841im  -0.808415+0.588613im        0.279452+0.96016im
   0.980437-0.196833im   0.380447+0.924803im      -0.0076575-0.999971im
   0.955999+0.293371im  -0.955126-0.296201im      -0.0458815+0.998947im

In [155]:
UU = OffsetArrays.no_offset_view(UU)

10×10×2 Array{ComplexF64, 3}:
[:, :, 1] =
 0.00163509+0.999999im   0.285763+0.9583im     …    0.857324-0.514777im
    0.48143+0.876485im  -0.982476+0.186389im       -0.697065+0.717008im
  -0.978167+0.207821im   0.604138+0.79688im        -0.985228+0.171246im
    0.78589-0.618367im  -0.380902+0.924615im       -0.570605+0.821225im
  -0.972366+0.23346im     0.32284-0.946454im        0.467586+0.883948im
  -0.760918-0.648848im   0.998838-0.0481857im  …   -0.970078-0.242794im
 -0.0914562-0.995809im  -0.853892-0.520451im        -0.25856-0.965995im
  -0.992672+0.120841im  -0.808415+0.588613im        0.279452+0.96016im
   0.980437-0.196833im   0.380447+0.924803im      -0.0076575-0.999971im
   0.955999+0.293371im  -0.955126-0.296201im      -0.0458815+0.998947im

[:, :, 2] =
 -0.921215-0.389054im    0.935704-0.352786im  …  -0.819825+0.572615im
 -0.165691+0.986178im   -0.615573-0.78808im      -0.408947+0.912558im
 0.0459272-0.998945im    0.238235+0.971207im      0.848795-0.528722im
  0.669975-0.742

In [156]:
F12 = calc_F12(UU)
sum(F12)/(2*pi*1im)

-2.518429760051489 + 8.661541324215172im

In [72]:
using NBInclude
@nbinclude("../Hofstadter Single Particle in Theta Space.ipynb")
using QuantumOptics
using LinearAlgebra

In [99]:
Nx=Ny=5
N=Nx*Ny
p=1
q=Ny
alpha=p/q
PN=2
N1 = 10
N2 = 10
dx=2*pi/N1
dy=dx
Tx=collect(range(start=0, stop=2*pi-dx, step=dx))
Ty=collect(range(start=0, stop=2*pi-dy, step=dy));

In [100]:
function Op_Sp(Nx, Ny, alpha, Tx, Ty)
    
    H_T = HSP_T(Nx, Ny, alpha, Tx, Ty, 0)
   
    N = Nx*Ny
    basis = NLevelBasis(N) 
    
    H = SparseOperator(basis)
    
    for m in 1:N
        for n in 1:N
            H = H + H_T[m,n] * transition(basis, m, n)
        end
    end
    
    return H
end

Op_Sp (generic function with 1 method)

In [101]:
op = Op_Sp(Nx, Ny, alpha, Tx[1], Ty[1]);

In [102]:
basis = NLevelBasis(N)
states = bosonstates(basis, PN)
function get_mb_op(PN, op)
    
    basis = NLevelBasis(N)
    states = bosonstates(basis, PN)
    basis_mb = ManyBodyBasis(basis, states)
    
    H = SparseOperator(basis_mb) 
    for m in 1:N
        for n in 1:N
            H += op.data[m,n] * transition(basis_mb, m, n)
        end
    end
    
    return H
end

get_mb_op (generic function with 1 method)

In [103]:
En, Un = eigen(dense(get_mb_op(PN, op)).data)
En

325-element Vector{Float64}:
 -5.932895978286749
 -5.932895978286747
 -5.932895978286745
 -5.932895978286745
 -5.932895978286745
 -5.932895978286745
 -5.932895978286745
 -5.932895978286744
 -5.932895978286744
 -5.932895978286744
 -5.9328959782867425
 -5.9328959782867425
 -5.932895978286742
  ⋮
  5.804226065180614
  5.804226065180614
  5.804226065180614
  5.804226065180614
  5.8042260651806155
  5.8042260651806155
  5.8042260651806155
  5.8042260651806155
  5.804226065180617
  5.804226065180617
  5.804226065180617
  5.804226065180619

In [104]:
# executeme

function Chern_Nums(Tx, Ty, n1, n2)
    
    Sum=0
    
    for tx in range(start=1, stop=length(Tx))
        for ty in range(start=1, stop=length(Ty))
            
            op = Op_Sp(Nx, Ny, 1/q, Tx[tx], Ty[ty])
            H_mb = get_mb_op(PN, op)
            w1, v1 = eigen(dense(H_mb).data)
            i = sortperm(w1, by=real);w1 = w1[i];v1 = v1[:,i]

            if isapprox(v1[1,1], 0+0im, atol=0.001) == false
                v1 = v1 ./ v1[1,1]
                v1 = v1 / norm(v1)
            end
            
            v1 = v1[:,n1:n2]
            
            #------------------------------------
            op = Op_Sp(Nx, Ny, 1/q, Tx[tx]+dx, Ty[ty])
            H_mb = get_mb_op(PN, op)
            w2, v2 = eigen(dense(H_mb).data)
            i = sortperm(w2, by=real);w2 = w2[i];v2 = v2[:,i]

            if isapprox(v2[1,1], 0+0im, atol=0.001) == false
                v2 = v2 ./ v2[1,1]
                v2 = v2 / norm(v2)
            end
            
            v2 = v2[:,n1:n2]
            
            #------------------------------------
            op = Op_Sp(Nx, Ny, 1/q, Tx[tx], Ty[ty]+dy)
            H_mb = get_mb_op(PN, op)
            w3, v3 = eigen(dense(H_mb).data)
            i = sortperm(w3, by=real);w3 = w3[i];v3 = v3[:,i]


            if isapprox(v3[1,1], 0+0im, atol=0.001) == false
                v3 = v3 ./ v3[1,1]
                v3 = v3 / norm(v3)
            end
            
            v3 = v3[:,n1:n2]
            
            #------------------------------------
            op = Op_Sp(Nx, Ny, 1/q, Tx[tx]+dx, Ty[ty]+dy)
            H_mb = get_mb_op(PN, op)
            w4, v4 = eigen(dense(H_mb).data)
            i = sortperm(w4, by=real);w4 = w4[i];v4 = v4[:,i]

            if isapprox(v4[1,1], 0+0im, atol=0.001) == false
                v4 = v4 ./ v4[1,1]
                v4 = v4 / norm(v4)
            end
            
            v4 = v4[:,n1:n2]
            
            #----------LINK VARIABLES------------
            U1=det(adjoint(v1)*v2)
            U1=U1/abs(U1)
            U2=det(adjoint(v2)*v4)
            U2=U2/abs(U2)
            U3=det(adjoint(v3)*v4)
            U3=U3/abs(U3)
            U4=det(adjoint(v1)*v3)
            U4=U4/abs(U4)
            
            #----------BERRY CURVATURE-----------
            F=log(U1*U2*1/U3*1/U4)
            Sum=Sum+F
            
        end
    end
    
    return 1/(2*pi*1im)*Sum
end

Chern_Nums (generic function with 1 method)

In [ ]:
for i in 1:Int(length(states)/Nx):length(states)
    C = Chern_Nums(Tx, Ty, i, i+(Int(length(states)/Nx))-1)
    println(C)
end